# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Análisis exploratorio y curación de datos

### Trabajo práctico entregable - Grupo 22 - Parte 1

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago

**Seguimiento:** Meinardi, Vanesa

---

## Librerías

In [ ]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
from sqlalchemy import create_engine, text

pd.set_option('display.max_rows', 1000) # cambiar el número de filas que se mostrarán usando display.max_rows.
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None  # default='warn'

sns.set_context('talk')
sns.set_theme(style='white')

## Lectura del dataset de sysarmy

In [ ]:
#datos kaggle
url_kag = 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv'
df_kag = pd.read_csv(url_kag)
#datos airbnb
url_airb = 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv'
df_airb = pd.read_csv(url_airb)
total_ans_kag = len(df_airb) # cantidad de respuestas en el dataset kaggle
total_ans_kag = len(df_airb) # cantidad de respuestas en el dataset airbnb

data source:
https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv

In [ ]:
engine = create_engine('sqlite:///sysarmy.sqlite3', echo=False)

#creo por un lado el df de kaggle
df_kag.to_sql('survey1', con=engine, if_exists="replace")

#creo por otro el df de airbnb
df_airb.to_sql('survey2', con=engine, if_exists="replace")
print(df_airb.keys())
#me quedo solo con el precio y codigo postal
#rel_col_airb = ['price', 'zipcode']
#df_airb2 = df_airb[rel_col_airb]

In [ ]:
#cantidad de registros por ciudad:
query_c = "SELECT price FROM survey2 GROUP BY city"
#solo por ciudad y barrio:
query_cb = "SELECT price FROM survey2 GROUP BY city and neighborhood"

con = engine.connect()
sql_text = text(query_cb)
result = con.execute(sql_text)


